<h1><center>Векторные представления слов</center></h1>



![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

Есть три сценария работы с векторными представлениями:
- Взять предобученную модель
- Обучить свою модель 
- Взять предобученную модель и дообучить ее


## Предобученные модели: RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятора семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный текстовый файл 
* .bin.gz — бинарный файл

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же векторы обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных моделей *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ : https://rusvectores.org/ru/models/

In [26]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/veronica/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
#!curl http://vectors.nlpl.eu/repository/20/220.zip -o ru_embeddings.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  608M  100  608M    0     0  9552k      0  0:01:05  0:01:05 --:--:-- 9865k      0  0:01:08  0:00:36  0:00:32  9.8M0:01:07  0:00:41  0:00:26 9973k:06  0:00:46  0:00:20 10.0M:01:05  0:01:01  0:00:04 10.0M


In [12]:
#!unzip ru_embeddings.zip

In [13]:
!ls ru_embeddings

alice.txt                 regular_expressions.ipynb test_data.csv
classification.ipynb      ru_analogy_tagged.txt     test_labels.csv
data                      ru_embeddings             text_preprocessing.ipynb
pipeline.png              ru_embeddings.zip         train_data.csv
recap_embeddings.ipynb    ruscorpora_mystem_cbow


In [14]:
model_path = 'ru_embeddings/model.bin'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [21]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **Важно!** В названиях моделей на `rusvectores` указано, какой набор тегов они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [22]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print(f'Увы, слова "{word}" нет в модели!')

день_NOUN
неделя_NOUN :  0.767143726348877
месяц_NOUN :  0.7355298399925232
утро_NOUN :  0.6559638381004333
час_NOUN :  0.632419228553772
ночь_NOUN :  0.5809687972068787
сутки_NOUN :  0.5776056051254272
вечер_NOUN :  0.5675091743469238
минута_NOUN :  0.5483242869377136
день»_PROPN :  0.5270609259605408
денька_NOUN :  0.5157712697982788


ночь_NOUN
вечер_NOUN :  0.7615348100662231
утро_NOUN :  0.7531793713569641
рассвет_NOUN :  0.727114737033844
полночь_NOUN :  0.6713059544563293
полдень_NOUN :  0.6476588845252991
сумерки_NOUN :  0.6017960906028748
утр_NOUN :  0.5851606130599976
день_NOUN :  0.5809689164161682
темнота_NOUN :  0.5586054921150208
ночной_ADJ :  0.5421528220176697


человек_NOUN
житель_NOUN :  0.6111521124839783
женщина_NOUN :  0.5880182385444641
мужчина_NOUN :  0.5467938780784607
душа_NOUN :  0.5095201134681702
более_ADV :  0.4962872266769409
население_NOUN :  0.476238489151001
человеческий_ADJ :  0.4728814661502838
солдат_NOUN :  0.4657357931137085
пациент_NOUN :  0.45008

In [23]:
model_ru['биткоин_NOUN']

KeyError: "Key 'биткоин_NOUN' not present"

Находим косинусную близость пары слов:

In [24]:
print(model_ru.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.26860568


Построим аналогии: 
* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*


Что получится, если вычесть из пиццы Италию и прибавить Россию?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [26]:
print(model_ru.most_similar(positive=['пицца_NOUN', 'россия_NOUN'], negative=['италия_NOUN'])[0][0])

хот-дог_NOUN


In [27]:
model_ru.doesnt_match('пицца_NOUN пельмень_NOUN хот-дог_NOUN ананас_NOUN'.split())

'ананас_NOUN'

## Предобученные модели: GloVe

Скачаем модель с сайта: https://nlp.stanford.edu/projects/glove/

In [30]:
#! wget https://nlp.stanford.edu/data/glove.6B.zip

In [7]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [10]:
! head -n 1 glove.6B.100d.txt

the -0.038194 -0.24487 0.72812 -0.39961 0.083172 0.043953 -0.39141 0.3344 -0.57545 0.087459 0.28787 -0.06731 0.30906 -0.26384 -0.13231 -0.20757 0.33395 -0.33848 -0.31743 -0.48336 0.1464 -0.37304 0.34577 0.052041 0.44946 -0.46971 0.02628 -0.54155 -0.15518 -0.14107 -0.039722 0.28277 0.14393 0.23464 -0.31021 0.086173 0.20397 0.52624 0.17164 -0.082378 -0.71787 -0.41531 0.20335 -0.12763 0.41367 0.55187 0.57908 -0.33477 -0.36559 -0.54857 -0.062892 0.26584 0.30205 0.99775 -0.80481 -3.0243 0.01254 -0.36942 2.2167 0.72201 -0.24978 0.92136 0.034514 0.46745 1.1079 -0.19358 -0.074575 0.23353 -0.052062 -0.22044 0.057162 -0.15806 -0.30798 -0.41625 0.37972 0.15006 -0.53212 -0.2055 -1.2526 0.071624 0.70565 0.49744 -0.42063 0.26148 -1.538 -0.30223 -0.073438 -0.28312 0.37104 -0.25217 0.016215 -0.017099 -0.38984 0.87424 -0.72569 -0.51058 -0.52028 -0.1459 0.8278 0.27062


In [15]:
import numpy as np

glove = {}
with open('./glove.6B/glove.6B.200d.txt', 'r') as f:
    for line in f:
        word, embedding = line.split(' ',1)
        wordEmbedding = np.array([float(value) for value in embedding[1:].split(' ')])
        glove[word] = wordEmbedding

print(len(glove))

400000


In [16]:
len(glove['hello'])

200

## Предобученные модели: fastText

FastText использует не только векторы слов, но и векторы n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Руководство](https://fasttext.cc/docs/en/support.html)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

На сайте проекта можно найти предобученные модели для 157 языков (в том числе русского): https://fasttext.cc/docs/en/crawl-vectors.html

In [1]:
import fasttext

In [1]:
ft = fasttext.load_model('cc.ru.300.bin')

In [18]:
ft['модель']

array([ 3.90226841e-02,  2.94893887e-02,  7.75389597e-02,  5.80755584e-02,
       -2.04796474e-02, -2.60778125e-02, -4.31554951e-02, -7.66336499e-03,
        7.35045224e-03,  8.39629862e-03, -3.03267990e-03,  6.59276769e-02,
        2.52222875e-03, -1.30695077e-02,  4.03538942e-02,  1.97116341e-02,
        7.49906823e-02,  3.26768160e-02, -1.30058741e-02,  5.22828884e-02,
        5.06116338e-02, -7.52337575e-02, -3.59434858e-02,  4.87469845e-02,
        1.21515250e-05,  2.44456176e-02, -6.05776981e-02,  3.70115340e-02,
        3.57847698e-02, -1.98335946e-02,  5.92223294e-02, -6.81746677e-02,
       -1.24129206e-02, -3.59787606e-02, -4.27804003e-03,  4.76003997e-02,
       -7.42428824e-02, -1.28765211e-01, -1.36825100e-01,  9.66134109e-03,
       -5.21074906e-02,  1.38343694e-02, -2.73327827e-02,  5.51195256e-02,
        2.06264984e-02,  5.95376752e-02,  2.22954024e-02,  7.45679426e-04,
       -5.00567667e-02,  4.27671038e-02, -2.68213451e-02,  1.15477806e-02,
       -2.37321910e-02,  

У fasttext есть все те же методы, что в gensim, но называются они иначе:

In [19]:
ft.get_nearest_neighbors('чай')

[(0.764227032661438, 'кофе'),
 (0.739784836769104, 'Чай'),
 (0.7071998119354248, 'чая'),
 (0.7018463015556335, 'чаи'),
 (0.6877091526985168, 'свежезаваренный'),
 (0.6864805221557617, 'напиток'),
 (0.6854358911514282, 'каркадэ'),
 (0.6772830486297607, 'чай.'),
 (0.667264461517334, 'чай-'),
 (0.6647352576255798, 'чаёк')]

In [20]:
ft.get_analogies("женщина", "мужчина", "актер")

[(0.8757159113883972, 'актриса'),
 (0.7068515419960022, 'артистка'),
 (0.694389820098877, 'киноактриса'),
 (0.6874823570251465, 'Актриса'),
 (0.6860095262527466, 'кинозвезда'),
 (0.6789741516113281, 'певица'),
 (0.6663230061531067, 'красавица-актриса'),
 (0.6603893637657166, 'актриса.'),
 (0.6569409966468811, 'Киноактриса'),
 (0.6488985419273376, 'актрисса')]

Важная особенность: так как модель обучена на символьных n-граммах, нет проблемы OOV (out of vocabulary) слов:

In [21]:
ft.get_nearest_neighbors('книжонка')

[(0.7441761493682861, 'книженция'),
 (0.7169104218482971, 'книжица'),
 (0.6966618895530701, 'брошюрка'),
 (0.6812918782234192, 'книжонку'),
 (0.6812120079994202, 'книжонок'),
 (0.6565592288970947, 'книжонки'),
 (0.6502901911735535, 'книжонке'),
 (0.6479876637458801, 'книжечка'),
 (0.6242943406105042, 'статейка'),
 (0.6214005351066589, 'книга-то')]

In [22]:
ft.get_nearest_neighbors('компютер')

[(0.7335842251777649, 'компъютер'),
 (0.7109572291374207, 'компютера'),
 (0.7098245024681091, 'компьютер'),
 (0.6971184015274048, 'компютерный'),
 (0.6874867081642151, 'копьютер'),
 (0.674636960029602, 'копм'),
 (0.6739663481712341, 'Компютер'),
 (0.6691707968711853, 'компутер'),
 (0.6670798063278198, 'компютеру'),
 (0.6656312346458435, 'комп')]

## Как обучить свою модель: word2vec

В качестве обучающих данных возьмем англоязычные отзывы о фильмах с сайта IMDB (данные взяты отсюда http://ai.stanford.edu/~amaas/data/sentiment/).

In [4]:
import pandas as pd

pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

In [18]:
import os

os.listdir('aclImdb/train/pos/')[:10]

['4715_9.txt',
 '12390_8.txt',
 '8329_7.txt',
 '9063_8.txt',
 '3092_10.txt',
 '9865_8.txt',
 '6639_10.txt',
 '10460_10.txt',
 '10331_10.txt',
 '11606_10.txt']

In [19]:
files = [f for f in os.listdir('aclImdb/train/pos/') if f.endswith('.txt')]
len(files)

12500

In [22]:
texts = []

for file in files:
    with open (f'aclImdb/train/pos/{file}', 'r') as text_file:
        text = text_file.read()
        texts.append(text)
        
data = pd.DataFrame.from_dict({'review': texts})
len(data)

12500

In [23]:
data.head()

,review
0,"For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan ""The Skipper"" Hale jr. as a police Sgt."
1,"Bizarre horror movie filled with famous faces but stolen by Cristina Raines (later of TV's ""Flamingo Road"") as a pretty but somewhat unstable model with a gummy smile who is slated to pay for her attempted suicides by guarding the Gateway to Hell! The scenes with Raines modeling are very well captured, the mood music is perfect, Deborah Raffin is charming as Cristina's pal, but when Raines moves into a creepy Brooklyn Heights brownstone (inhabited by a blind priest on the top floor), things really start cooking. The neighbors, including a fantastically wicked Burgess Meredith and kinky couple Sylvia Miles & Beverly D'Angelo, are a diabolical lot, and Eli Wallach is great fun as a wily police detective. The movie is nearly a cross-pollination of ""Rosemary's Baby"" and ""The Exorcist""--but..."
2,"A solid, if unremarkable film. Matthau, as Einstein, was wonderful. My favorite part, and the only thing that would make me go out of my way to see this again, was the wonderful scene with the physicists playing badmitton, I loved the sweaters and the conversation while they waited for Robbins to retrieve the birdie."
3,"It's a strange feeling to sit alone in a theater occupied by parents and their rollicking kids. I felt like instead of a movie ticket, I should have been given a NAMBLA membership.<br /><br />Based upon Thomas Rockwell's respected Book, How To Eat Fried Worms starts like any children's story: moving to a new town. The new kid, fifth grader Billy Forrester was once popular, but has to start anew. Making friends is never easy, especially when the only prospect is Poindexter Adam. Or Erica, who at 4 1/2 feet, is a giant.<br /><br />Further complicating things is Joe the bully. His freckled face and sleeveless shirts are daunting. He antagonizes kids with the Death Ring: a Crackerjack ring that is rumored to kill you if you're punched with it. But not immediately. No, the death ring unleas..."
4,"You probably all already know this by now, but 5 additional episodes never aired can be viewed on ABC.com I've watched a lot of television over the years and this is possibly my favorite show, ever. It's a crime that this beautifully written and acted show was canceled. The actors that played Laura, Whit, Carlos, Mae, Damian, Anya and omg, Steven Caseman - are all incredible and so natural in those roles. Even the kids are great. Wonderful show. So sad that it's gone. Of course I wonder about the reasons it was canceled. There is no way I'll let myself believe that Ms. Moynahan's pregnancy had anything to do with it. It was in the perfect time slot in this market. I've watched all the episodes again on ABC.com - I hope they all come out on DVD some day. Thanks for reading."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приведем все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [24]:
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")

In [27]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [28]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer=tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [33]:
sentences = [review_to_sentences(review) for review in data["review"].values]

print(len(sentences))
print(*sentences[:3])

12500
[['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem'], ['imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny'], ['maureen', 'stapleton', 'is', 'a', 'scene', 'stealer'], ['the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream'], ['watch', 'for', 'alan', 'the', 'skipper', 'hale', 'jr', 'as', 'a', 'police', 'sgt']] [['bizarre', 'horror', 'movie', 'filled', 'with', 'famous', 'faces', 'but', 'stolen', 'by', 'cristina', 'raines', 'later', 'of', 'tv', 's', 'flamingo', 'road', 'as', 'a', 'pretty', 'but', 'somewhat', 'unstable', 'model', 'with', 'a', 'gummy', 'smile', 'who', 'is', 'slated', 'to', 'pay', 'for', 'her', 'attempted', 'suicides', 'by', 'guarding', 'the', 'gateway', 'to', 'hell'], ['the', 'scenes', 'with', 'raines', 'modeling', 'are', 'very', 'well', 'captured', 'the', 'mood', 'music', 'is', 'perfect', 'deborah', 'raffin', 'is', 'charming', 'as', 'cr

In [34]:
flat_sentences = [item for sublist in sentences for item in sublist]
len(flat_sentences)

132111

In [35]:
# это понадобится нам позже для обучения модели fasttext

with open('clean_text.txt', 'w') as f:
    for s in flat_sentences:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* vector_size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**Важно!** Обратите внимание, что тренировка модели не включает предобработку! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [38]:
print("Training model...")

%time model_en = word2vec.Word2Vec(flat_sentences, workers=4, vector_size=100, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 22.2 s, sys: 159 ms, total: 22.3 s
Wall time: 6.19 s


Смотрим, сколько в модели слов.

In [41]:
print(len(model_en.wv.vectors))

13761


Попробуем оценить модель вручную, аналогично тому, как проверяли предобученные модели.

In [42]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))

[('actress', 0.8907668590545654)]


In [43]:
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

[('men', 0.6068826913833618)]


In [44]:
print(model_en.wv.most_similar("usa", topn=3))

[('italy', 0.8183871507644653), ('europe', 0.8170506954193115), ('japan', 0.7882201671600342)]


In [45]:
print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

novel


### Почему важно, на каких данных обучалась модель?

Посмотрим на ближайшие по смыслу слова к слову "star":
- в модели, обученной на обзорах фильмов
- в модели, обученной на Википедии

In [38]:
print(*model_en.wv.most_similar("star", topn=10), sep='\n')

('stars', 0.6092053651809692)
('hudson', 0.4575274586677551)
('singer', 0.446683406829834)
('stardom', 0.4426514506340027)
('fame', 0.4368395209312439)
('starred', 0.41369011998176575)
('studded', 0.4064015746116638)
('starring', 0.4028608202934265)
('tyrone', 0.4026487469673157)
('icon', 0.3995055556297302)


In [39]:
model_en.similarity('star', 'celebrity')

0.38832128

In [40]:
model_en.similarity('star', 'sky')

0.16640092

In [41]:
model_en.similarity('star', 'shine')

0.20186296

Скачаем предобученную модель fastText для английского:

In [42]:
import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore') 

ft_eng = fasttext.load_model('cc.en.300.bin')

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
cosine_similarity([ft_eng['star']], [ft_eng['celebrity']])

array([[0.44439566]], dtype=float32)

In [45]:
cosine_similarity([ft_eng['star']], [ft_eng['sky']])

array([[0.29363436]], dtype=float32)

In [46]:
cosine_similarity([ft_eng['star']], [ft_eng['shine']])

array([[0.26306346]], dtype=float32)

### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [46]:
model_en.wv.similarity('white', 'rabbit')

0.43382952

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [47]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [48]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

Saving model...


In [49]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(93627, 150225)

"Белый" и "кролик" стали ближе друг к другу!

In [50]:
model.wv.similarity('white', 'rabbit')

0.44453424

## Как обучить свою модель: fastText

Выше мы записали предобработанные данные IMDB в текстовый файл, теперь мы можем использовать его для обучения модели fastText.

In [51]:
import fasttext

# так можно обучить свою модель 
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

AttributeError: module 'fasttext' has no attribute 'train_unsupervised'

In [53]:
ft_model.get_word_vector("movie")

array([ 0.10647972,  0.00348743,  0.00400553, -0.2220917 ,  0.02774876,
        0.28113014,  0.04588589, -0.00953584, -0.14305185,  0.06663238,
       -0.01932244,  0.06766095, -0.06964523, -0.00879823, -0.2754987 ,
        0.00865352, -0.02323557, -0.25205076, -0.12951218,  0.03342189,
        0.00371615, -0.05362495,  0.01820608,  0.1001416 , -0.00959393,
       -0.18463574,  0.3780635 , -0.02545302, -0.18444885,  0.12707062,
       -0.10905846,  0.16443293,  0.15409584, -0.11409819,  0.24385491,
       -0.00680395, -0.17201523, -0.11899467, -0.1555182 , -0.09888287,
       -0.0370929 ,  0.07110037, -0.18011254, -0.13228868,  0.06374579,
        0.11771804,  0.12609613, -0.0567237 ,  0.18499018,  0.1786773 ,
        0.14431344, -0.04145625, -0.05425412, -0.19320773, -0.00719959,
       -0.13159607, -0.15365084, -0.0127184 ,  0.0429236 , -0.0468537 ,
       -0.16655543,  0.14787091,  0.00640271,  0.10557158,  0.3033813 ,
        0.21118031,  0.0810559 , -0.15322076,  0.05768304,  0.03

In [54]:
ft_model.get_nearest_neighbors('actor')

[(0.6824725270271301, 'charactor'),
 (0.6627753973007202, 'ctor'),
 (0.6476548314094543, 'actors'),
 (0.630966067314148, 'tractor'),
 (0.5709237456321716, 'actress'),
 (0.5643734931945801, 'reactor'),
 (0.5564367771148682, 'benefactor'),
 (0.5310477018356323, 'hector'),
 (0.5144302845001221, 'contractor'),
 (0.5066649913787842, 'factor')]

In [55]:
ft_model.get_analogies("woman", "man", "actor")

[(0.6606210470199585, 'actress'),
 (0.5363813042640686, 'actresses'),
 (0.48164188861846924, 'ctor'),
 (0.4782077372074127, 'actors'),
 (0.46812912821769714, 'charactor'),
 (0.4576756954193115, 'seductress'),
 (0.4522528052330017, 'tractor'),
 (0.4366052448749542, 'hector'),
 (0.42155134677886963, 'womaniser'),
 (0.40410301089286804, 'abductor')]

OOV:

In [56]:
ft_model.get_nearest_neighbors('actr')

[(0.7510600090026855, 'actresses'),
 (0.747826099395752, 'actress'),
 (0.6713401675224304, 'actors'),
 (0.6301003098487854, 'actor'),
 (0.50783771276474, 'acting'),
 (0.4529079794883728, 'xd'),
 (0.4442768394947052, 'gzsz'),
 (0.44143927097320557, 'act'),
 (0.4292294681072235, 'acharya'),
 (0.42825278639793396, 'nb')]

In [57]:
ft_model.get_nearest_neighbors('moviegeek')

[(0.6236038208007812, 'eek'),
 (0.612317681312561, 'movie'),
 (0.5866702795028687, 'geek'),
 (0.5764365792274475, 'moviegoer'),
 (0.571036696434021, 'moviemaking'),
 (0.5661671161651611, 'movies'),
 (0.5481372475624084, 'creek'),
 (0.5346364974975586, 'moviegoers'),
 (0.5314581394195557, 'cq'),
 (0.5229887366294861, 'bwp')]

## Оценка

Мы научились обучать модели, научились загружать готовые, а как понять, какая модель лучше? Или вот, например, мы обучили модель, а как понять, насколько она хорошая? Рассмотрим два метода: с помощью метрик, основанных на подготовленных данных, и визуализации.

Для этого существуют специальные наборы данных для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Сибирь и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного набора данных мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Набор данных будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Данные для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [58]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

In [59]:
!less ru_analogy_tagged.txt

: capital-common-countries
афины_S греция_S багдад_S ирак_S
афины_S греция_S бангкок_S таиланд_S
афины_S греция_S пекин_S китай_S
афины_S греция_S берлин_S германия_S
афины_S греция_S берн_S швейцария_S
афины_S греция_S каир_S египет_S
афины_S греция_S канберра_S австралия_S
афины_S греция_S ханой_S вьетнам_S
афины_S греция_S гавана_S куба_S
афины_S греция_S хельсинки_S финляндия_S
афины_S греция_S исламабад_S пакистан_S
афины_S греция_S кабул_S афганистан_S
афины_S греция_S лондон_S англия_S
афины_S греция_S мадрид_S испания_S
афины_S греция_S москва_S россия_S
афины_S греция_S осло_S норвегия_S
афины_S греция_S оттава_S канада_S
афины_S греция_S париж_S франция_S
афины_S греция_S рим_S италия_S
афины_S греция_S стокгольм_S швеция_S
афины_S греция_S тегеран_S иран_S
афины_S греция_S токио_S япония_S
багдад_S ирак_S бангкок_S таиланд_S
:

In [60]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


In [61]:
print(model_ru.most_similar(positive=['мальчик_S', 'бабка_S'], negative=['девочка_S'])[0][0])

бабушка_S


In [62]:
len(res[4]['incorrect']), len(res[4]['correct'])

(88, 218)

## Визуализация

На полученную модель можно посмотреть, визуализировав ее, например, на плоскости.
### t-SNE

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).


Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [54]:
model_path = "movie_reviews.model"

model = word2vec.Word2Vec.load(model_path)

In [61]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(flat_sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = [model.wv[word] for word in top_words]

  0%|          | 0/132111 [00:00<?, ?it/s]

['the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i', 'that', 'this', 's', 'as', 'with', 'for', 'was', 'film', 'but', 'movie', 'his', 'on', 'you', 'he', 'are', 'not', 't', 'one', 'have', 'be', 'by', 'all', 'who', 'an', 'at', 'from', 'her', 'they', 'has', 'so', 'like', 'about', 'very', 'out', 'there', 'she', 'what', 'or', 'good', 'more', 'when']


In [62]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 1min 25s, sys: 3.39 s, total: 1min 28s
Wall time: 2min 4s


In [63]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD, и потом применять t-SNE:

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

In [ ]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)